In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
datafile = pd.read_excel('/content/drive/MyDrive/Sem 6/ME308/Datafile-2022.xlsx')

In [ ]:
datafile = pd.read_excel("Datafile-2022.xlsx")
datafile = datafile.iloc[:48,:]

def get_player_data(player_name):
    player_name = player_name.upper()
    player = datafile.loc[datafile["player_name"] == player_name]
    if len(player) < 1:
        print("Player not found")
        return
    lowest_price = player.iloc[0, 1]
    highest_price = player.iloc[0, 2]
    if lowest_price == highest_price:
        highest_price = 2 * lowest_price
    rating = player.iloc[0, 3]
    type = player.iloc[0, 4]
    return {"player_name": player_name, "highest_price": highest_price, "lowest_price": lowest_price, "rating": rating,
            "type": type}

def evaluate_team(team):
    return sum(player["rating"] for player in team)







In [ ]:
class Auctioneer:
    def __init__(self, model):
        if model == "linear":
            self.model = self.linear_model
        elif model == "exponential":
            self.model = self.exponential_model
        elif model == "quadratic":
            self.model = self.quadratic_model
        elif model == "quartic":
            self.model = self.fourth_power_model
        else:
            raise ValueError(f'{model} not found')
        self.data = datafile

    def linear_model(self, bid_price, lowest_price, highest_price):
        if bid_price >= highest_price:
            return 1
        if bid_price <= lowest_price:
            return 0
        return (bid_price - lowest_price) / (highest_price - lowest_price)

    def exponential_model(self, bid_price, lowest_price, highest_price):
        if bid_price >= highest_price:
            return 1
        if bid_price <= lowest_price:
            return 0
        return (np.exp(bid_price) - np.exp(lowest_price)) / (np.exp(highest_price) - np.exp(lowest_price))

    def quadratic_model(self, bid_price, lowest_price, highest_price):
        if bid_price >= highest_price:
            return 1
        if bid_price <= lowest_price:
            return 0
        return (bid_price ** 2 - lowest_price ** 2) / (highest_price ** 2 - lowest_price ** 2)

    def fourth_power_model(self, bid_price, lowest_price, highest_price, n=4):
        if bid_price >= highest_price:
            return 1
        if bid_price <= lowest_price:
            return 0
        return (bid_price ** n - lowest_price ** n) / (highest_price ** n - lowest_price ** n)

    def response_to_bid(self, player_name, bid_price):
        player_data = get_player_data(player_name)
        yes_prob = self.model(bid_price, player_data["lowest_price"], player_data["highest_price"])
        return np.random.choice([1, 0], p=[yes_prob, 1 - yes_prob])


In [ ]:
class Auctioneer:
    def __init__(self, model):
        if model == "linear":
            self.model = self.linear_model
        elif model == "exponential":
            self.model = self.exponential_model
        elif model == "quadratic":
            self.model = self.quadratic_model
        else:
            raise ValueError(f'{model} not found')
        self.data = datafile
        self.data = self.data.iloc[:, 0:5]

    def linear_model(self, bid_price, lowest_price, highest_price):
        if bid_price>=highest_price:
            return 1
        if bid_price<=lowest_price:
            return 0
        return (bid_price - lowest_price) / (highest_price - lowest_price)

    def exponential_model(self, bid_price, lowest_price, highest_price):
        if bid_price>=highest_price:
            return 1
        if bid_price<=lowest_price:
            return 0
        return (math.exp(bid_price) - math.exp(lowest_price)) / (math.exp(highest_price) - math.exp(lowest_price))

    def quadratic_model(self, bid_price, lowest_price, highest_price):
        if bid_price>=highest_price:
            return 1
        if bid_price<=lowest_price:
            return 0
        return (bid_price**2 - lowest_price**2) / (highest_price**2 - lowest_price**2)

    def response_to_bid(self, player_name, bid_price):
        player_name = player_name.upper()
        player = self.data.loc[self.data["player_name"] == player_name]
        if len(player) < 1:
            print("Player not found")
            return
        lowest_price = player.iloc[0,3]
        highest_price = player.iloc[0, 4]
        if lowest_price == highest_price:
          highest_price = 2*lowest_price #########################################
        yes_prob = self.model(bid_price, lowest_price, highest_price)
        return np.random.choice([1, 0], p=[yes_prob, 1 - yes_prob])


In [ ]:
class Bidder:
    def __init__(self, auctioneer, money_const=200, num_players=11):
        self.num_players = num_players
        self.money = 50  # initial cash
        self.min_change = 0.25
        self.team = []
        self.auctioneer = auctioneer
        self.money_const = money_const * self.num_players
        # self.additional_utility_const = 1

    def cal_max_price(self, player_data):
        return self.money / self.money_const * player_data["rating"]

    def bid(self, player_name):
        player_data = get_player_data(player_name)
        # utility = self.cal_utility(player_data)
        max_price = self.cal_max_price(player_data)
        price = player_data["lowest_price"]
        while price <= max_price:
            if price > self.money:
                # print(f"Cannot bid further. Price to bid: {price} crs, money left: {self.money}")
                break
            response = self.auctioneer.response_to_bid(player_name, price)
            if response == 1:
                player_data["price"] = price
                self.money -= price
                self.team.append(player_data)
                # print(f"{player_name.lower().capitalize()} is sold at {price}."
                #       f" Calculated max price: {max_price} crs, rating:{player_data['rating']}")
                break
            price += self.min_change


    def auction(self):
        for player_name in datafile["player_name"]:
            if self.money<=0:
                break
            self.bid(player_name)
            if len(self.team) >= self.num_players:
                break
        return self.team


In [ ]:
players = [("AIDEN MARKRAM", 9.96),
           ("JOSH HAZLEWOOD", 9.58),
           ("DEVON CONWAY", 9.36),
           ("WANINDU HASARANGA", 9.25),
           ("RASSIE VAN DER DUSSEN", 9.13),
           ("QUINTON DE KOCK", 8.94),
           ("TIM SOUTHEE", 8.88),
           ("NICHOLAS POORAN", 8.88),
           ("JASON ROY", 8.67),
           ("SHREYAS IYER", 8.65),
           ("MITCHELL SANTNER", 8.65)]

auctioneer = Auctioneer("quartic")
bidder = Bidder(auctioneer, money_const=215)

solution = []

for player in players:
    response = bidder._bid(player[0], player[1])
    if response==None:
        print(f"{player[0]} was not sold")
    else:
        print(f"{player[0]} was sold at {response}")

In [ ]:

auctioneer = Auctioneer("exponential")
bidder = Bidder(auctioneer)
team = bidder.auction()
print(team)
print(evaluate_team(team))



In [1]:
# lowest_price = 5
# highest_price = 15
# x = np.linspace(lowest_price,highest_price,endpoint=True)
# y_linear = (x - lowest_price) / (highest_price - lowest_price)
# y_exp = (np.exp(x) - np.exp(lowest_price)) / (np.exp(highest_price) - np.exp(lowest_price))
# y_quad = (x ** 2 - lowest_price ** 2) / (highest_price ** 2 - lowest_price ** 2)
# y_cubic = (x ** 4 - lowest_price ** 4) / (highest_price ** 4 - lowest_price ** 4)
# plt.plot(x,y_linear)
# plt.plot(x,y_exp)
# plt.plot(x,y_quad)
# plt.plot(x,y_cubic)
# plt.legend(["linear", "exponential", "quadratic", "quartic"])
# plt.xlabel("Price (crs)")
# plt.ylabel("Probability")